In [35]:
!pip install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
!pip install transformers requests beautifulsoup4 pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# had to uninstall tokenizers and reinstall tokenizers (in env) to get to work

In [38]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


In [39]:
tokens = tokenizer.encode('I hated this. It was not good ', return_tensors='pt')

In [40]:
result = model(tokens)

In [41]:
torch.argmax(result.logits)

tensor(0)

In [42]:
result.logits

tensor([[ 4.0000,  2.5746,  0.1991, -2.6189, -3.3137]],
       grad_fn=<AddmmBackward0>)

In [43]:
int(torch.argmax(result.logits))+1

1

In [44]:
r = requests.get('https://www.yelp.com/biz/captain-stoker-monterey')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

# *comment* will also pull replies from business owners or other non-reviewers

In [45]:
reviews[4]

"This honestly is up there with top oatmilk lattes I've had. Do try it! I've had this coffee shop on my list of places to check out for a long time now and finally got around to it. The friendly staff, welcoming and cute environment, and great drink all add to the experience! I'm definitely coming back to get another latte! This looks like a nice place to sit down and get some work done as well, as there is seating available. I also saw some nice merch that was being sold!"

In [46]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,** Very smooth and refreshing tasting Iced Van...,5
1,The best vanilla latte and avocado toast! I st...,5
2,First time here during a Labor Day weekend tri...,4
3,"Everytime I take a Monterey day trip, I make s...",5
4,This honestly is up there with top oatmilk lat...,5
5,"Great food and drinks, and really indoor space...",5
6,"Fresh home made vanilla lattes, delicious.!Ver...",5
7,I can't wait for go back! We stopped yesterday...,5
8,"You can't beat their vanilla lattes, you just ...",4
9,A good coffee shop in Monterey Drinks: they ar...,4
